In [52]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from operator import itemgetter
import os
import copy

%matplotlib inline 

def GraphGenerator_AutoLabel(rects, ax, location):
    """
    Helper function
    """
    horizontalAlign = 'center'
    if location == "up":
        verticalAlign='bottom'
    else:
        verticalAlign='top'

    for rect in rects:
        h = rect.get_height()
        label = "%.2f" % (h)
        if location == "up":
            h = h*1.0
        else:
            h = h*1.0
        ax.text(rect.get_x()+0.1, h, label,
                ha=horizontalAlign, va=verticalAlign)

def DrawBarChart(Inputs, Title, Ymax = None, Ymin = None):
    """
    Draw the bar chart
    """
    '''
    Prepare chart input
    '''
    X_label = ["Total Runtime", "Tensorflow Runtime", "Instrumentation Runtime"]
    Passes = ['Pass=' + str(i) for i in range(1, 35)]
    X_label.extend(Passes)
    

    """
    
        
    NameList = []
    Speedup = []
    for item in Inputs:
        NameList.append(item[0])
        Speedup.append(item[1])

    x = NameList
    y = Speedup
    '''
    Draw bar chart
    '''
    plt.rc('font', size=2.8)
    fig = plt.figure(dpi=600)
    width = 0.25 # the width of the bars

    # Split into N bar chart
    splitCount = 3
    step = len(x) // splitCount
    patch_speedup = mpatches.Patch(color='green', label='Speedup By ABC')
    percentage = 0.33

    for i in range(splitCount):
        indexStart = i*step
        indexEnd = indexStart + int(percentage * len(x))

        x_sub = x[indexStart:indexEnd]
        y_sub = y[indexStart:indexEnd]
        #x_sub_order = [x for x in range(indexStart, indexEnd)]
        index = np.arange(len(x_sub))
        ax = fig.add_subplot(splitCount, 1 , i+1)

        rects = ax.bar(index, y_sub, width, color='green')
        # Do we need to limit the y axis?
        if Ymin == None and Ymax != None:
            plt.ylim(ymax=Ymax)
        elif Ymin != None and Ymax == None:
            plt.ylim(ymin=Ymin)
        elif Ymin != None and Ymax != None:
            plt.ylim(ymin=Ymin, ymax=Ymax)

        y_label_loc = "up"
        GraphGenerator_AutoLabel(rects, ax, y_label_loc)

        XLength = len(x)
        ax.set_ylabel('Execution Time From ABC Relative to Clang -O3: {}% - {}%'.format(
            round((indexStart/XLength)*100), round((indexEnd/XLength)*100)))
        
        ax.set_xticks(index)
        ax.set_xticklabels(x_sub, rotation=270)

    fig.legend( handles=[patch_speedup] )
    fig.suptitle(Title, fontweight='bold', fontsize=10, y=1.0)
    plt.tight_layout()
    plt.show()
    fig.savefig(Title + ".png", dpi=fig.dpi, bbox_inches='tight')
    """

def PrintStat(Inputs):
    Dict = dict.fromkeys(list(Inputs.keys()), [])
    for key, info in Dict.items():
        tmp = []
        total = Inputs[key]['TotalRuntime']
        tf = Inputs[key]['TensorflowRuntime']
        instr = Inputs[key]['TotalPassInstrumentationTime']
        tmp.append(total/total)
        tmp.append(tf/total)
        tmp.append(instr/total)
        for i in range(1, 35):
            usage = Inputs[key][str(i)]
            tmp.append(usage/instr)

        print('{}: tf={}, instr={}, total={}'.format(key, tf, instr, total))
    
def ProcessRawData():
    # read raw data into dict
    loc = 'raw-data/BuildTimeAnalysis'
    for root, dirs, files in os.walk(loc):
        for File in files:
            key = File.split('-')[0]
            RawDict[key] = {}
            with open(os.path.join(root, File), 'r') as f:
                for line in f:
                    line = line.strip()
                    # e.g. PassID:1, Time:37
                    if line.startswith('PassID'):
                        info = line.split(',')
                        PassID = int(info[0].split(':')[1])
                        time = int(info[1].split(':')[1])
                        if RawDict[key].get(PassID) is None:
                            RawDict[key][PassID] = 0
                        RawDict[key][PassID] += time
                    # e.g. tensorflow-predict-time:2388 
                    elif line.startswith('tensorflow-predict-time'):
                        if RawDict[key].get('TensorflowRuntime') is None:
                            RawDict[key]['TensorflowRuntime'] = 0
                        RawDict[key]['TensorflowRuntime'] += int(line.split(':')[1])
                    # e.g. all-runtime:98323
                    elif line.startswith('all-runtime'):
                        if RawDict[key].get('TotalRuntime') is None:
                            RawDict[key]['TotalRuntime'] = 0
                        RawDict[key]['TotalRuntime'] += int(line.split(':')[1])
    with open('BuildTimeAnalysis.json', 'w') as js:                                                                        
        json.dump(RawDict, js)
    
if __name__ == '__main__':
    RawDict = {}
    '''
    Normally, you don't need to comment out the following to process the raw data.
    "BuildTimeAnalysis.json" is already processed.
    '''
    #ProcessRawData()
    
    RawDict = json.load(open("BuildTimeAnalysis.json"))
    ProcessedDict = copy.deepcopy(RawDict)
    # generate the total instrumentation time
    for target, info in RawDict.items():
        for key, count in info.items():
            if key != 'TotalRuntime' and key != 'TensorflowRuntime':
                if ProcessedDict[target].get('TotalPassInstrumentationTime') is None:
                    ProcessedDict[target]['TotalPassInstrumentationTime'] = 0
                ProcessedDict[target]['TotalPassInstrumentationTime'] += count
    
    PrintStat(ProcessedDict)
    DrawBarChart(Inputs=ProcessedDict, Title="ABC Runtime Analysis", Ymax = 1.2, Ymin = 0)

aha: tf=453810, instr=1354066, total=2232695
kc: tf=93283779, instr=480687481, total=727331489
siod: tf=11059878, instr=31025535, total=49683424
viterbi: tf=344518, instr=1887889, total=2588511
sgefa: tf=467990, instr=4988250, total=6392330
hexxagon: tf=1686766, instr=8880768, total=12074174
spiff: tf=2267994, instr=9550053, total=13942083
burg: tf=4370363, instr=17515469, total=25927074
Obsequi: tf=1908176, instr=16948034, total=21514364
clamscan: tf=22418614, instr=271177990, total=341815648
sqlite3: tf=24991782, instr=190325366, total=265172262
lua: tf=16156525, instr=53084904, total=81018514
hbd: tf=3188443, instr=29797020, total=37113615
oggenc: tf=9936155, instr=90015091, total=125358396
treecc: tf=6812983, instr=42700660, total=57497086
lemon: tf=2546743, instr=25022883, total=32256594
SPASS: tf=95880070, instr=218816670, total=354678931
make_dparser: tf=7281225, instr=64684710, total=83264943
lambda: tf=3762053, instr=25293048, total=32669508
ldecod: tf=9981233, instr=172537697